# Projeto final de aprendizado de máquina - classificador de imagens

**Estudante:** João Gabriel de Oliveira Bicalho

**Matrícula:** 2017015134

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from PIL import Image

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
from sklearn.ensemble import AdaBoostClassifier
import seaborn as sns

## Introdução

O objetivo deste trabalho é criar um classificador para dois datasets de imagens. O primeiro dataset (Chinese MNIST) contém imagens de 15 números chineses manuscritos. Já o segundo (Fashion MNIST) contém imagens de 10 classes de produtos de moda. O objetivo do classificador é atribuir corretamente essas labels às imagens.

Neste trabalho serão implementados dois classificadores: o primeiro utilizando um método de boosting na imagem achatada, e o segundo utilizando uma CNN _(Convolutional Neural Network)_. O intuito é utilizar a mesma CNN para ambos os datasets, apenas adequando o tamanho das camadas de entrada e saída. No final os resultados dos dois classificadores serão comparados.

# Datasets

## Chinese MNIST

Este dataset possui 15000 images de 64x64 pixels em escala de cinzas representando números em chinês. No total, o dataset possui 15 classes a serem categorizadas, e cada imagem possui apenas uma delas. As classes são mostradas na tabela abaixo, que apresenta os ideograma a serem identificados e os seus respectivos valores no sistema numérico decimal:

In [ ]:
dfc = pd.read_csv('/kaggle/input/chinese-mnist/chinese_mnist.csv')
images = np.array([np.asarray(Image.open('/kaggle/input/chinese-mnist/data/data/input_%d_%d_%d.jpg'%(x['suite_id'], x['sample_id'], x['code']))) for x in dfc.iloc])
pd.DataFrame(zip([9,10,100,1000,10000,100000000,0,1,2,3,4,5,6,7,8],dfc['character'].unique()), columns=['valor', 'caractere']).sort_values(by='valor')

## Fashion MNIST

Este dataset possui 60000 images de treino e 10000 imagens de teste, cada uma com 28x28 pixels em escala de cinzas representando vestimentas ou acessórios de moda. Cada imagem está associada a uma classe. Existem 10 classes dentro deste dataset, que são mostradas na tabela abaixo (coluna valor), junto com a sua label associada (coluna classe):

In [ ]:
pd.DataFrame([[0, 'T-shirt/top'],
[1, 'Trouser'],
[2, 'Pullover'],
[3, 'Dress'],
[4, 'Coat'],
[5, 'Sandal'],
[6, 'Shirt'],
[7, 'Sneaker'],
[8, 'Bag'],
[9, 'Ankle boot']], columns=['classe', 'valor'])

## Tratamento dos dados

Após o carregamento de cada um dos datasets, fez-se o tratamento de seus dados. As imagens de entrada foram normalizadas de forma que o valor de cada pixel esteja entre 0.0 e 1.0. Em seguida, foi atribuído uma label numérica para cada uma das classes de cada dataset. Para o dataset _Fashion MNIST_ estas labels já vieram do dataset. Para o dataset _Chinese MNIST_ as labels foram extraídas usando a classe _LabelEncoder_ do sklearn. Para o classificador com CNN as labels foram depois codificadas em um vetor categórico usando a técnica de one-hot-encoding (isto é, um vetor de dimensão igual ao número de classes com todas as posições zeradas, exceto a que representa sua classe, que tem 1).

In [ ]:
# Normalização e etiquetação Chinese MNIST

xc = images/255.0
xc = xc.reshape((-1, 64, 64, 1)) # Apenas para ficar no formado que a CNN prefere

yc = np.array(dfc['value'])
lec = preprocessing.LabelEncoder()
yc_int=lec.fit_transform(yc)
yc=to_categorical(yc_int)
n_cclasses=len(yc[0])

In [ ]:
# Carregamento e normalização Fashion MNIST

data_train = pd.read_csv('../input/fashionmnist/fashion-mnist_train.csv')
data_test = pd.read_csv('../input/fashionmnist/fashion-mnist_test.csv')

img_rows, img_cols = 28, 28
input_shape = (img_rows, img_cols, 1)

xf_train = np.array(data_train.iloc[:, 1:], dtype='float32')/255.0
yf_train_int = np.array(data_train.iloc[:, 0])
yf_train = to_categorical(yf_train_int)

xf_test = np.array(data_test.iloc[:, 1:], dtype='float32')/255.0
yf_test_int = np.array(data_test.iloc[:, 0])
yf_test = to_categorical(yf_test_int)

xf_train = xf_train.reshape((len(xf_train), 28, 28, 1))
xf_test = xf_test.reshape((len(xf_test), 28, 28, 1))

## Separação de conjuntos de treino e teste

Após o preprocessamento dos dados, o dataset _Chinese MNIST_ foi dividido em uma parte para treino e outra para teste. Foi decidido utilizar 20% dos dados para o teste e o restante para o treino. O dataset _Fashion MNIST_ já veio separado nestes dois conjuntos.

In [ ]:
xc_train, xc_test, yc_train, yc_test, yc_train_int, yc_test_int = train_test_split(xc, yc, yc_int, test_size=0.2, random_state=8)

# Treinamento dos modelos para o _Chinese MNIST_

## AdaBoosting

A primeira abordagem utilizada foi utilizar uma técnica de boosting para a classificação. Foram testados alguns algoritmos e, por fim, foi decidido utilizar o AdaBoosting, já que os outros facilmente ficavam com overfitting. Para usar este classificador, a imagem foi achatada em um vetor de $64*64=4096$ posições. 

Através de experimentação foi decidido definir o número de estimadores usados pelo AdaBoosting como 50, pois o algoritmo ficava melhor com a adição de mais estimadores. Para validação foi utilizado k-fold com k=5.

Os resultados são apresentados logo após as células de implementação.

In [ ]:
xc_train_ada = xc_train.reshape((len(xc_train), -1))
xc_test_ada = xc_test.reshape((len(xc_test), -1))

xc_ada_fold = KFold(n_splits=5, random_state=9, shuffle=True)

In [ ]:
ada = AdaBoostClassifier(n_estimators=50)

results = []
acc=[]
for train_index, val_index in xc_ada_fold.split(xc_train_ada):
    print('Fold %d...' % (len(results)+1))
    my_x_train = xc_train_ada[train_index]
    my_y_train = yc_train_int[train_index]
    my_x_val = xc_train_ada[val_index]
    my_y_val = yc_train_int[val_index]
    
    ada.fit(my_x_train, my_y_train)
    my_y_pred = ada.predict(my_x_val)
    prfs = precision_recall_fscore_support(my_y_val, my_y_pred, average='micro')
    facc = accuracy_score(my_y_val, my_y_pred)
    results.append(prfs[:-1])
    acc.append(facc)

print('Concluído. Média dos resultados no treino:')
print('precisão, revocação, fscore')
results = np.array(results).mean(axis=0)
print(results)
print('acurácia: ', np.array(acc).mean());

In [ ]:
ada.fit(xc_train_ada, yc_train_int)
yc_pred_ada = ada.predict(xc_test_ada)
c_prfs_ada = precision_recall_fscore_support(yc_test_int, yc_pred_ada, average='micro')
c_acc_ada = accuracy_score(yc_test_int, yc_pred_ada)
print('Resultados no teste')
print('precisão, revocação, fscore')
print(c_prfs_ada[:-1])
print('acurácia: ', c_acc_ada);

### Resultados

Como é possível notar na saída das últimas duas células, os resultados nos conjuntos de treino e teste ficaram bem parecidos, e ambos ficaram ruins, apresentando uma acurácia em torno de 20%. Apesar disto, o resultado ficou dentro do esperado, já que o dado de entrada (imagem) é um dado não estruturado, e classificadores geralmente não funcionam bem nesse tipo de dado.

## CNN

Em busca de tentar obter um modelo com maior acurácia foi criada uma Rede Neural Convolucional. Esse tipo de rede geralmente obtém melhores resultados com imagens, já que ela consegue trabalhar com dados de mais de uma dimensão e, através de convoluções, consegue extrair informações dos pixels considerando localidade espacial.

Conforme já mencionado, para treinar esta rede as labels associadas às imagens foram codificadas como um vetor one-hot-encoding.

A rede criada recebe uma entrada de dimensão (64, 64, 1) e possui 2 camadas de convolução com 16 filtros (3, 3) cada, seguida por uma camada de MaxPooling (2, 2), uma camada de Dropout (com 50% de probabilidade), mais 2 camadas de convolução com 16 filtros (3, 3) cada, mais uma camada de Dropout (0.5), uma camada de achatamento e uma camada densa com 15 perceptrons para saída do vetor de classificação. Em todas as camadas de convolução foi utilizada a função de ativação ReLU, já na camada densa de saída foi utilizado softmax.

As camadas de Max Pooling foram introduzidas para diminuir a dimensionalidade da saída das camadas anteriores através de um down-sampling. Dessa forma é possível fazer assunções melhores sobre as características contidas nas subregiões agregadas pelo pooling. Já as camadas de Dropout tentam evitar que o modelo criado sofra de overfitting, desabilitando aleatoriamente partes das outras camadas da rede.

Foi utilizado o otimizador Adam, o loss foi calculado através de entropia cruzada categórica, e a métrica de avaliação foi a acurácia.


In [ ]:
modelc = models.Sequential()
modelc.add(layers.Conv2D(16, (3, 3), activation='relu', input_shape=(64, 64, 1)))
modelc.add(layers.Conv2D(16, (3, 3), activation='relu'))
modelc.add(layers.MaxPooling2D((2, 2)))
modelc.add(layers.Dropout(0.5))
modelc.add(layers.Conv2D(16, (3, 3), activation='relu'))
modelc.add(layers.Conv2D(16, (3, 3), activation='relu'))
modelc.add(layers.Dropout(0.5))
modelc.add(layers.Flatten())
modelc.add(layers.Dense(n_cclasses, activation='softmax'))
modelc.summary()

modelc.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

O modelo da rede neural foi treinado utilizando 50 épocas e dividindo o dado de treino em treino e validação. Esta divisão dos dados foi realizada automaticamente pelo Keras no momento do _fit_ , como pode ser visto na célula abaixo.

In [ ]:
cepochs = 50
historyc = modelc.fit(xc_train, yc_train, epochs=cepochs, batch_size=32,
                    validation_split=0.2)

In [ ]:
plt.plot(list(range(cepochs)), historyc.history['loss'], label='treino')
plt.plot(list(range(cepochs)), historyc.history['val_loss'], label='validação')
plt.legend()
plt.title('loss')
plt.xlabel('número da época')
plt.show()
plt.plot(list(range(cepochs)), historyc.history['accuracy'], label='treino')
plt.plot(list(range(cepochs)), historyc.history['val_accuracy'], label='validação')
plt.title('accurácia')
plt.xlabel('número da época')
plt.legend()
plt.show()

### Resultados

Os resultados obtidos da CNN foram muito bons, obtendo uma ótima acurácia (próxima de 100%) tanto no conjunto de treino quanto no conjunto de validação, e também mantendo a diminuição do loss durante o treinamento. O modelo não parece apresentar overfitting.
Após a finalização da mudança dos parâmetros e a geração dos gráficos acima, foi realizado o teste do modelo no conjunto de teste, que manteve uma excelente acurácia, conforme pode ser observado abaixo.

In [ ]:
yc_pred_test = modelc.predict(xc_test)

yc_pred_test_argmax = np.argmax(yc_pred_test, axis=1)
print('acurácia no teste: ', accuracy_score(yc_test_int, yc_pred_test_argmax))

#### Exemplos de classificação correta

Na imagem abaixo é possível ver vários exemplos de classificações corretas do classificador. No título de cada sub-figura tem-se a previsão realizada.

In [ ]:
meus_x = xc_test[yc_pred_test_argmax == yc_test_int].reshape((-1,64,64))
meus_y = lec.inverse_transform(yc_test_int[yc_pred_test_argmax == yc_test_int])

fig=plt.figure(figsize=(8, 8))
columns = 5
rows = 5
for i in range(1, columns*rows +1):
    fig.add_subplot(rows, columns, i)
    plt.imshow(meus_x[i])
    plt.title(meus_y[i])
    plt.axis('off')
plt.subplots_adjust(hspace=0.25)
plt.show()

#### Exemplos de classificação incorreta

Na imagem abaixo é possível ver vários exemplos de classificações incorretas do classificador. No título de cada sub-figura tem-se a previsão realizada e a correta, sendo que a correta está entre parênteses.

In [ ]:
print('Valores preditos (valores reais)')
meus_x = xc_test[yc_pred_test_argmax != yc_test_int].reshape((-1,64,64))
meus_y = lec.inverse_transform(yc_pred_test_argmax[yc_pred_test_argmax != yc_test_int])
reais_y = lec.inverse_transform(yc_test_int[yc_pred_test_argmax != yc_test_int])

fig=plt.figure(figsize=(8, 8))
columns = 5
rows = 5
for i in range(1, columns*rows +1):
    fig.add_subplot(rows, columns, i)
    plt.imshow(meus_x[i])
    plt.title("%d (%d)" % (meus_y[i], reais_y[i]))
    plt.axis('off')
plt.subplots_adjust(hspace=0.25, wspace=0.5)
plt.show()

# Treinamento dos modelos para o _Fashion MNIST_

Diante dos resultados do dataset anterior, para este dataset apenas foi treinado o modelo CNN, já que ele obteve uma acurácia muito maior que o AdaBoosting.

## CNN

Para treinar a CNN para o _Fashion MNIST_ foi utilizada a mesma arquitetura da CNN do _Chinese MNIST_ , com exceção do tamanho da camada de entrada e de saída, que foram definidas de acordo com os dados deste dataset: (28, 28, 1) e 10, respectivamente.

In [ ]:
nf_classes = len(yf_train[0])
modelf = models.Sequential()
modelf.add(layers.Conv2D(16, (3, 3), activation='relu', input_shape=(28, 28, 1)))
modelf.add(layers.Conv2D(16, (3, 3), activation='relu'))
modelf.add(layers.MaxPooling2D((2, 2)))
modelf.add(layers.Dropout(0.5))
modelf.add(layers.Conv2D(16, (3, 3), activation='relu'))
modelf.add(layers.Conv2D(16, (3, 3), activation='relu'))
modelf.add(layers.Dropout(0.5))
modelf.add(layers.Flatten())
modelf.add(layers.Dense(nf_classes, activation='softmax'))
modelf.summary()

modelf.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

O modelo da rede neural foi treinado utilizando 50 épocas e dividindo o dado de treino em treino e validação. Esta divisão dos dados foi realizada automaticamente pelo Keras no momento do _fit_ , como pode ser visto na célula abaixo.

In [ ]:
fepochs = 50
historyf = modelf.fit(xf_train, yf_train, epochs=fepochs, batch_size=32,
                    validation_split=0.2, shuffle=True)

In [ ]:
plt.plot(list(range(fepochs)), historyf.history['loss'], label='treino')
plt.plot(list(range(fepochs)), historyf.history['val_loss'], label='teste')
plt.legend()
plt.title('loss')
plt.show()
plt.plot(list(range(fepochs)), historyf.history['accuracy'], label='treino')
plt.plot(list(range(fepochs)), historyf.history['val_accuracy'], label='teste')
plt.title('accuracy')
plt.legend()
plt.show()

### Resultados

Os resultados obtidos da CNN foram muito bons, obtendo uma ótima acurácia (próxima de 90%) tanto no conjunto de treino quanto no conjunto de validação, e também mantendo a diminuição do loss durante o treinamento. O modelo não parece apresentar overfitting.
Após a finalização da mudança dos parâmetros e a geração dos gráficos acima, foi realizado o teste do modelo no conjunto de teste, que manteve uma excelente acurácia, conforme pode ser observado abaixo.

In [ ]:
yf_pred_test = modelf.predict(xf_test)

yf_pred_test_argmax = np.argmax(yf_pred_test, axis=1)
print('acurácia no teste: ', accuracy_score(yf_test_int, yf_pred_test_argmax))

In [ ]:
my_lef = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

#### Exemplos de classificação correta

Na imagem abaixo é possível ver vários exemplos de classificações corretas do classificador. No título de cada sub-figura tem-se a previsão realizada.

In [ ]:
meus_x = xf_test[yf_pred_test_argmax == yf_test_int].reshape((-1,28,28))
meus_y = yf_test_int[yf_pred_test_argmax == yf_test_int]

fig=plt.figure(figsize=(8, 8))
columns = 5
rows = 5
for i in range(1, columns*rows +1):
    fig.add_subplot(rows, columns, i)
    plt.imshow(meus_x[i])
    plt.title(my_lef[meus_y[i]])
    plt.axis('off')
plt.subplots_adjust(hspace=0.25)
plt.show()

#### Exemplos de classificação incorreta

Na imagem abaixo é possível ver vários exemplos de classificações incorretas do classificador. No título de cada sub-figura tem-se a previsão realizada e a correta, sendo que a correta está entre parênteses.

In [ ]:
print('Valores preditos (valores reais)')
meus_x = xf_test[yf_pred_test_argmax != yf_test_int].reshape((-1,28,28))
meus_y = yf_pred_test_argmax[yf_pred_test_argmax != yf_test_int]
reais_y = yf_test_int[yf_pred_test_argmax != yf_test_int]

fig=plt.figure(figsize=(8, 8))
columns = 5
rows = 5
for i in range(1, columns*rows +1):
    fig.add_subplot(rows, columns, i)
    plt.imshow(meus_x[i])
    plt.title("%s\n(%s)" % (my_lef[meus_y[i]], my_lef[reais_y[i]]))
    plt.axis('off')
plt.subplots_adjust(hspace=0.6, wspace=0.5)
plt.show()

# Discussão dos resultados


Como pôde ser observado, a rede neural convolucional performou muito melhor do que o método de boosting durantes os testes no dataset Chinese MNIST, possuindo uma acurácia muito maior. Isso ajuda a demonstrar a capacidade de CNNs para classificação de imagens, além de mostrar que classificadores clássicos realmente não trabalham bem com dados não estruturados. Além disso, foi possível conferir que a arquitetura de CNN apresentada funcionou adequadamente para os dois datasets testados, não apresentando overfitting e tendo ótima acurácia. O uso de uma rede relativamente pequena e de dropouts provavelmente colaborou para que isso fosse possível.

Além disso, as entradas serem bem comportadas (apenas uma coisa na imagem, mesma dimensão e bem consistentes) também foi excelente para fazer com que o classificador obtivesse acurácia elevada.

Nas imagens abaixo é possível ver as matrizes de confusão do classificador AdaBoosting e CNN para o dataset _Chinese MNIST_ , e do CNN para o _Fashion MNIST_ , nesta ordem. É possível perceber que os classificadores CNN conseguem separar as classes muito bem, ao passo que o AdaBoosting não é bom, se confundindo muito em quase todas as classes.



In [ ]:
# Matriz de confusão dataset Chinese MNIST adaboosting
cf_matrix=confusion_matrix(yc_test_int, yc_pred_ada)
fig, ax = plt.subplots(figsize=(12,10))
sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=True, 
            fmt='.2%', cmap='Blues')
plt.title('Matriz de confusão das previsões no conjunto de teste do dataset Chinese MNIST (AdaBoosting)')
plt.show()

In [ ]:
# Matriz de confusão dataset Chinese MNIST cnn
cf_matrix=confusion_matrix(yc_test_int, yc_pred_test_argmax)
fig, ax = plt.subplots(figsize=(12,10))
sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=True, 
            fmt='.2%', cmap='Blues')
plt.title('Matriz de confusão das previsões no conjunto de teste do dataset Chinese MNIST (CNN)')
plt.show()

In [ ]:
# Matriz de confusão dataset Fashion MNIST
cf_matrix=confusion_matrix(yf_test_int, yf_pred_test_argmax)
fig, ax = plt.subplots(figsize=(8,7))
sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=True, 
            fmt='.2%', cmap='Blues')
plt.title('Matriz de confusão das previsões no conjunto de teste do dataset Fashion MNIST')
plt.show()

# Conclusão

Foi possível implementar adequadamente todos os classificadores, assim como compará-los. Os classificadores CNN obtiveram ótima performance, enquanto o classificador AdaBoosting teve uma performance ruim. Em geral, os resultados foram satisfatórios e dentro do esperado.